In [1]:
from google.colab import drive
drive.mount('/content/drive')
from pathlib import Path
import os
repo_path = Path.cwd()/'drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/notebooks/'
os.chdir(str(repo_path))

Mounted at /content/drive


In [2]:
!cp -r /content/drive/MyDrive/calcification-detection-project/calcification_detecion/data_rois.zip /home/
!unzip /home/data_rois.zip -d /home
!mv /home/home/vzalevskyi/projects/data_rois /home/data_rois
!rm -r /home/home

^C
Archive:  /home/data_rois.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /home/data_rois.zip or
        /home/data_rois.zip.zip, and cannot find /home/data_rois.zip.ZIP, period.
mv: cannot stat '/home/home/vzalevskyi/projects/data_rois': No such file or directory
rm: cannot remove '/home/home': No such file or directory


In [3]:
from pathlib import Path
thispath = Path.cwd().resolve()
import yaml

In [ ]:
# cfg_path = str(thispath.parent.parent/'deep_learning/config.yml')
# with open(cfg_path, "r") as ymlfile:
#     cfg = yaml.safe_load(ymlfile)

In [4]:
cfg = {
    'model': {
        'activation': 'LeakyReLU',
        'dropout': 0.2,
        'fc_dims': [512, 512],
        'freeze_weights': True,
        'backbone': 'resnet18',
        'pretrained': True
    },
    'dataset': {
        'extract_patches': False,
        'delete_previous': False,
        'extract_patches_method': 'all',
        'patch_size': 224,
        'stride': 100,
        'min_breast_fraction_roi': 0.5,
        'n_jobs': -1,
        'cropped_imgs': True,
        'ignore_diameter_px': 15,
        'patch_images_path': '/home/data_rois/',
        'train_neg_to_pos_ratio': 5,
        'normalization': 'z_score'
    },
    'dataloaders': {
        'train_batch_size': 64,
        'val_batch_size': 256
    },
    'data_aug': {
        'prob': 0
    },
    'training': {
        'criterion': 'BCEWithLogitsLoss',
        'optimizer': 'SGD',
        'optimizer_args': {
            'lr': 0.0001, 'momentum': 0.9
        },
        'lr_scheduler': 'StepLR',
        'lr_scheduler_args': {
            'step_size': 10, 'gamma': 0.1
        },
        'n_epochs': 20,
        'best_metric': 'f1_score',
        'resume_training': False,
        'early_stopping': True,
        'early_stopping_args':{
            'min_diff': 0.0001,
            'max_epoch': 3
        }
        },
    'experiment_name': 'freezed_resnet18_01'
}

In [5]:
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)

In [6]:
from pathlib import Path
thispath = Path.cwd().resolve()
import sys; sys.path.insert(0, str(thispath.parent))

from deep_learning.dataset.dataset import INBreast_Dataset_pytorch
from deep_learning.models.base_classifier import CNNClasssifier
import deep_learning.dl_utils as dl_utils

import logging
import torch
import time
import random
import yaml

import numpy as np
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T

from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

logging.basicConfig(level=logging.INFO)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def identity_function(arg):
    return arg


def train_model(datasets, dataloaders, data_transforms, model, criterion, optimizer, scheduler, cfg):

    # guarantee reproducibility
    since = time.time()
    random.seed(0)
    torch.manual_seed(1442)
    np.random.seed(0)

    # holders for best model
    best_metric = 0.0
    best_metric_name = cfg['training']['best_metric']

    exp_path = Path.cwd().parent.parent/f'data/deepl_runs/{cfg["experiment_name"]}'
    exp_path.mkdir(exist_ok=True, parents=True)
    best_model_path = exp_path / f'{cfg["experiment_name"]}.pt'
    chkpt_path = exp_path / f'{cfg["experiment_name"]}_chkpt.pt'

    if cfg['training']['resume_training']:
        checkpoint = torch.load(chkpt_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        init_epoch = checkpoint['epoch'] + 1
    else:
        init_epoch = 0

    # tensorboard loggs
    log_dir = exp_path/'tensorboard'
    log_dir.mkdir(exist_ok=True, parents=True)
    writer = SummaryWriter(log_dir=log_dir)

    early_stopping_count = 0
    previous_metric = 0

    for epoch in range(init_epoch, cfg['training']['n_epochs']):
        logging.info(f'Epoch {epoch+1}/{cfg["training"]["n_epochs"]}')
        logging.info(('-' * 10))

        # resample used negatives to use the large diversity of them that we have
        datasets['train'].update_sample_used(epoch)
        dataloaders['train'] = DataLoader(
            datasets['train'], batch_size=cfg['dataloaders']['train_batch_size'],
            shuffle=True, num_workers=4, drop_last=False)

        for phase in ['train', 'val']:
            # Set model to the corresponding mode and update lr if necessary
            if phase == 'train':
                if epoch != 0:
                    scheduler.step()
                writer.add_scalar(f"LearningRate/{phase}", scheduler.get_last_lr()[0], epoch)
                model.train()
            else:
                model.eval()

            # define holders for losses, preds and labels
            running_loss = 0.0
            epoch_preds, epoch_labels = [], []

            # Iterate over data.
            n_data = len(dataloaders[phase])
            for it, sample in tqdm(enumerate(dataloaders[phase]), total=n_data):

                # Apply transformations and send to device
                sample['img'] = data_transforms[phase](sample['img'])
                inputs = sample['img'].to(device)
                labels = sample['label'].to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward pass (track history if only in train)
                with torch.set_grad_enabled(phase == 'train'):
                    # predict
                    outputs = model(inputs)

                    # store values
                    epoch_preds.append(np.asarray(
                        torch.sigmoid(outputs.detach()).flatten().cpu()))
                    epoch_labels.append(np.asarray(labels.detach().cpu()))

                    # finish the comp. graph
                    loss = criterion(outputs.flatten(), labels.float())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # once in a while store the images batch to check
                if it in [25, 50, 100]:
                    imgs = T.functional.rgb_to_grayscale(sample['img']).cpu()
                    writer.add_images(f'Images/{phase}', imgs, epoch)
                    del imgs

                # get the epoch loss cumulatively
                running_loss += loss.item() * inputs.size(0)

            # compute and log the metrics for the epoch
            epoch_preds = np.concatenate(epoch_preds)
            epoch_labels = np.concatenate(epoch_labels)
            epoch_loss = running_loss / len(epoch_preds)
            metrics = dl_utils.get_metrics(epoch_labels, epoch_preds)
            dl_utils.tensorboard_logs(writer, epoch_loss, epoch, metrics, phase)

            # print status
            epoch_f1 = metrics['f1_score']
            message = f'{phase} Loss: {epoch_loss:.4f} Acc: {metrics["accuracy"]:.4f}' \
                f' F1: {epoch_f1:.4f} AUROC: {metrics["auroc"]:.4f}'
            logging.info(message)

            # save last and best checkpoint
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss}, chkpt_path)

            if phase == 'val':
                if metrics[best_metric_name] > best_metric:
                    best_metric = epoch_f1
                    best_threshold = metrics['threshold']
                    torch.save({
                        'model_state_dict': model.state_dict(),
                        'metrics': metrics,
                        'configuration': cfg
                        }, best_model_path)
                
                if cfg['training']['early_stopping']:
                    diff = metrics[best_metric_name] - previous_metric
                    if diff < cfg['training']['early_stopping_args']['min_diff']:
                        early_stopping_count += 1
                    else:
                        early_stopping_count = 0
                previous_metric = metrics[best_metric_name]

        if cfg['training']['early_stopping']:
            max_epochs = cfg['training']['early_stopping_args']['max_epoch']
            if early_stopping_count == max_epochs:
                msg = f'Early stopping after {max_epochs} epochs without' \
                    f' significant change in val metric'
                logging.info(msg)
                break
        logging.info(('-' * 10))

    time_elapsed = time.time() - since
    message = f'Training complete in {(time_elapsed // 60):.0f}m ' \
        f'{(time_elapsed % 60):.0f}s'
    logging.info(message)
    logging.info(f'Best val {best_metric_name}: {best_metric:4f}, threshold {best_threshold:.4f}')

    # close the tensorboard session
    writer.flush()
    writer.close()

    # load best model weights before returning
    best_model = torch.load(best_model_path)
    model.load_state_dict(best_model['model_state_dict'])
    return model


def main():
    # read the configuration file
    config_path = thispath.parent/"deep_learning/config.yml"
    with open(config_path, "r") as ymlfile:
        cfg = yaml.safe_load(ymlfile)

    # use the configuration for the dataset
    dataset_arguments = cfg['dataset']
    dataset_arguments['patch_images_path'] = Path(dataset_arguments['patch_images_path'])
    datasets = {
        'train': INBreast_Dataset_pytorch(
            partitions=['train'], neg_to_pos_ratio=dataset_arguments['train_neg_to_pos_ratio'],
            balancing_seed=0, **dataset_arguments),
        'val': INBreast_Dataset_pytorch(
            partitions=['validation'], neg_to_pos_ratio=None, **dataset_arguments)
    }

    # use the configuration for the dataloaders
    dataloaders = {
        'val': DataLoader(
            datasets['val'], batch_size=cfg['dataloaders']['train_batch_size'],
            num_workers=4, drop_last=False),
        'train': DataLoader(
            datasets['train'], batch_size=cfg['dataloaders']['val_batch_size'],
            shuffle=True, num_workers=4, drop_last=False)
    }

    # use the configuration for the transformations
    transforms = nn.Sequential(
        T.ColorJitter(brightness=0.4, contrast=0.4, saturation=0, hue=0),
        T.RandomAffine(
            degrees=(0, 20), translate=None, scale=None, shear=(1, 10, 1, 10),
            interpolation=T.InterpolationMode.BILINEAR, fill=0
        ),
        T.RandomPerspective(distortion_scale=0.2),
        T.RandomRotation(degrees=(0, 20)),
        T.RandomRotation(degrees=(90, 110)),
        T.RandomResizedCrop(size=(224, 224), scale=(0.9, 1), ratio=(1, 1)),
        T.RandomAutocontrast(),
        T.RandomHorizontalFlip(),
        T.RandomVerticalFlip()
    )
    transforms = T.RandomApply(transforms=transforms, p=cfg['data_aug']['prob'])
    data_transforms = {
        'train': identity_function if (cfg['data_aug']['prob'] == 0) else transforms,
        'val': identity_function
    }

    # model configs
    model = CNNClasssifier(
        activation=getattr(nn, cfg['model']['activation'])(),
        dropout=cfg['model']['dropout'],
        fc_dims=cfg['model']['fc_dims'],
        freeze_weights=cfg['model']['freeze_weights'],
        backbone=cfg['model']['backbone'],
        pretrained=cfg['model']['pretrained'],
    )
    model = model.model.to(device)

    # training configs
    criterion = getattr(nn, cfg['training']['criterion'])()

    optimizer = getattr(optim, cfg['training']['optimizer'])
    optimizer = optimizer(model.parameters(), **cfg['training']['optimizer_args'])

    scheduler = getattr(lr_scheduler, cfg['training']['lr_scheduler'])
    scheduler = scheduler(optimizer, **cfg['training']['lr_scheduler_args'])

    # train the model
    train_model(datasets, dataloaders, data_transforms, model, criterion, optimizer, scheduler, cfg)


# if __name__ == '__main__':
    # main()


In [ ]:
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 410/410 [00:41<00:00,  9.88it/s]
INFO:root:train Loss: 0.5325 Acc: 0.7126 F1: 0.2276 AUROC: 0.5364
100%|██████████| 297/297 [00:31<00:00,  9.57it/s]
INFO:root:val Loss: 0.2793 Acc: 0.6301 F1: 0.1581 AUROC: 0.5992
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 410/410 [00:41<00:00,  9.89it/s]
INFO:root:train Loss: 0.4524 Acc: 0.6749 F1: 0.3322 AUROC: 0.6357
100%|██████████| 297/297 [00:30<00:00,  9.71it/s]
INFO:root:val Loss: 0.2649 Acc: 0.5822 F1: 0.1756 AUROC: 0.6542
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 410/410 [00:41<00:00,  9.82it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['experiment_name'] = 'freezed_resnet18_02'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 410/410 [00:42<00:00,  9.76it/s]
INFO:root:train Loss: 0.4309 Acc: 0.7479 F1: 0.3959 AUROC: 0.6935
100%|██████████| 297/297 [00:30<00:00,  9.58it/s]
INFO:root:val Loss: 0.2536 Acc: 0.7119 F1: 0.2225 AUROC: 0.7178
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 410/410 [00:41<00:00,  9.84it/s]
INFO:root:train Loss: 0.3613 Acc: 0.7300 F1: 0.4530 AUROC: 0.7754
100%|██████████| 297/297 [00:30<00:00,  9.73it/s]
INFO:root:val Loss: 0.2374 Acc: 0.7371 F1: 0.2359 AUROC: 0.7322
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 410/410 [00:41<00:00,  9.89it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'min_max'
cfg['experiment_name'] = 'freezed_resnet18_03'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 410/410 [00:42<00:00,  9.56it/s]
INFO:root:train Loss: 0.4122 Acc: 0.6924 F1: 0.3947 AUROC: 0.7068
100%|██████████| 297/297 [00:30<00:00,  9.70it/s]
INFO:root:val Loss: 0.2574 Acc: 0.7583 F1: 0.2449 AUROC: 0.7230
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 410/410 [00:41<00:00,  9.81it/s]
INFO:root:train Loss: 0.3605 Acc: 0.7906 F1: 0.4810 AUROC: 0.7738
100%|██████████| 297/297 [00:30<00:00,  9.58it/s]
INFO:root:val Loss: 0.2447 Acc: 0.7720 F1: 0.2526 AUROC: 0.7335
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 410/410 [00:42<00:00,  9.73it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['experiment_name'] = 'freezed_resnet18_04'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 724/724 [01:07<00:00, 10.76it/s]
INFO:root:train Loss: 0.2768 Acc: 0.7942 F1: 0.3203 AUROC: 0.7267
100%|██████████| 297/297 [00:30<00:00,  9.68it/s]
INFO:root:val Loss: 0.2323 Acc: 0.7132 F1: 0.2232 AUROC: 0.7254
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 724/724 [01:07<00:00, 10.79it/s]
INFO:root:train Loss: 0.2443 Acc: 0.7809 F1: 0.3529 AUROC: 0.7856
100%|██████████| 297/297 [00:31<00:00,  9.53it/s]
INFO:root:val Loss: 0.2229 Acc: 0.7537 F1: 0.2430 AUROC: 0.7324
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 724/724 [01:06<00:00, 10.91it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 5
cfg['experiment_name'] = 'freezed_resnet18_05'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 410/410 [00:47<00:00,  8.67it/s]
INFO:root:train Loss: 0.3965 Acc: 0.7584 F1: 0.4333 AUROC: 0.7385
100%|██████████| 297/297 [00:35<00:00,  8.38it/s]
INFO:root:val Loss: 0.2556 Acc: 0.7654 F1: 0.2543 AUROC: 0.7402
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 410/410 [00:44<00:00,  9.32it/s]
INFO:root:train Loss: 0.3440 Acc: 0.7588 F1: 0.4797 AUROC: 0.7890
100%|██████████| 297/297 [00:31<00:00,  9.53it/s]
INFO:root:val Loss: 0.2172 Acc: 0.7066 F1: 0.2278 AUROC: 0.7366
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 410/410 [00:42<00:00,  9.65it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 5
cfg['dataloaders']['train_batch_size'] = 128
cfg['experiment_name'] = 'freezed_resnet18_06'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 205/205 [00:50<00:00,  4.08it/s]
INFO:root:train Loss: 0.4221 Acc: 0.7348 F1: 0.4192 AUROC: 0.7235
100%|██████████| 149/149 [00:39<00:00,  3.80it/s]
INFO:root:val Loss: 0.2660 Acc: 0.6982 F1: 0.2291 AUROC: 0.7352
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 205/205 [00:49<00:00,  4.14it/s]
INFO:root:train Loss: 0.3416 Acc: 0.7852 F1: 0.4948 AUROC: 0.7954
100%|██████████| 149/149 [00:39<00:00,  3.81it/s]
INFO:root:val Loss: 0.2285 Acc: 0.7864 F1: 0.2627 AUROC: 0.7433
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 205/205 [00:49<00:00,  4.13it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 5
cfg['model']['backbone'] = 'resnet34'
cfg['dataloaders']['train_batch_size'] = 64
cfg['experiment_name'] = 'freezed_resnet34_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 410/410 [00:57<00:00,  7.08it/s]
INFO:root:train Loss: 0.4023 Acc: 0.7490 F1: 0.4259 AUROC: 0.7315
100%|██████████| 297/297 [00:42<00:00,  6.93it/s]
INFO:root:val Loss: 0.2398 Acc: 0.7802 F1: 0.2526 AUROC: 0.7307
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 410/410 [00:58<00:00,  7.00it/s]
INFO:root:train Loss: 0.3463 Acc: 0.7974 F1: 0.5029 AUROC: 0.7975
100%|██████████| 297/297 [00:42<00:00,  7.03it/s]
INFO:root:val Loss: 0.2331 Acc: 0.7264 F1: 0.2363 AUROC: 0.7368
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 410/410 [00:58<00:00,  7.04it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 5
cfg['model']['backbone'] = 'resnet50'
cfg['dataloaders']['train_batch_size'] = 64
cfg['experiment_name'] = 'freezed_resnet50_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 410/410 [01:34<00:00,  4.32it/s]
INFO:root:train Loss: 0.3532 Acc: 0.7625 F1: 0.4703 AUROC: 0.7798
100%|██████████| 297/297 [01:06<00:00,  4.48it/s]
INFO:root:val Loss: 0.2145 Acc: 0.8058 F1: 0.3014 AUROC: 0.7893
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 410/410 [01:34<00:00,  4.34it/s]
INFO:root:train Loss: 0.3080 Acc: 0.8296 F1: 0.5605 AUROC: 0.8363
100%|██████████| 297/297 [01:06<00:00,  4.49it/s]
INFO:root:val Loss: 0.1967 Acc: 0.7640 F1: 0.2808 AUROC: 0.7967
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 410/410 [01:34<00:00,  4.35it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 5
cfg['model']['backbone'] = 'resnet101'
cfg['dataloaders']['train_batch_size'] = 64
cfg['experiment_name'] = 'freezed_resnet101_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 410/410 [02:31<00:00,  2.71it/s]
INFO:root:train Loss: 0.3693 Acc: 0.7290 F1: 0.4440 AUROC: 0.7621
100%|██████████| 297/297 [01:44<00:00,  2.84it/s]
INFO:root:val Loss: 0.2345 Acc: 0.7455 F1: 0.2618 AUROC: 0.7678
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 410/410 [02:31<00:00,  2.71it/s]
INFO:root:train Loss: 0.3278 Acc: 0.7960 F1: 0.5187 AUROC: 0.8155
100%|██████████| 297/297 [01:44<00:00,  2.84it/s]
INFO:root:val Loss: 0.2066 Acc: 0.7595 F1: 0.2752 AUROC: 0.7793
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 410/410 [02:31<00:00,  2.71it/s]
INFO:root:train Loss: 0.3127 Acc: 0.8068 F1: 0.5389 AUROC: 0.8347
100%|██████████| 297/297 [01:44<00:00,  2.84it/s]
INFO:root:val Loss: 0.2213 Acc: 0.8231 F1: 0.3068 AUROC: 0.7767
INFO:root:----------
INFO:root:Epoch 4/20
INFO:root:----------
100%|██████████| 410/410 [02:31<00:00,  2.71it/s]
INFO:root:train Loss: 0.3075 

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 5
cfg['model']['backbone'] = 'resnet152'
cfg['dataloaders']['train_batch_size'] = 64
cfg['experiment_name'] = 'freezed_resnet152_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 410/410 [03:27<00:00,  1.97it/s]
INFO:root:train Loss: 0.3531 Acc: 0.7914 F1: 0.4899 AUROC: 0.7833
100%|██████████| 297/297 [02:21<00:00,  2.09it/s]
INFO:root:val Loss: 0.2112 Acc: 0.7962 F1: 0.2929 AUROC: 0.7888
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 410/410 [03:28<00:00,  1.97it/s]
INFO:root:train Loss: 0.3041 Acc: 0.8108 F1: 0.5542 AUROC: 0.8434
100%|██████████| 297/297 [02:22<00:00,  2.09it/s]
INFO:root:val Loss: 0.1962 Acc: 0.7407 F1: 0.2659 AUROC: 0.7956
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 410/410 [03:28<00:00,  1.97it/s]
INFO:root:train Loss: 0.2905 Acc: 0.8320 F1: 0.5814 AUROC: 0.8570
100%|██████████| 297/297 [02:22<00:00,  2.09it/s]
INFO:root:val Loss: 0.2123 Acc: 0.8108 F1: 0.3112 AUROC: 0.7975
INFO:root:----------
INFO:root:Epoch 4/20
INFO:root:----------
100%|██████████| 410/410 [03:27<00:00,  1.97it/s]
INFO:root:train Loss: 0.2847 

In [ ]:
### Unfreezed experiments

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 5
cfg['model']['backbone'] = 'resnet18'
cfg['model']['freeze_weights'] = False
cfg['dataloaders']['train_batch_size'] = 64
cfg['experiment_name'] = 'resnet18_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 410/410 [01:42<00:00,  4.00it/s]
INFO:root:train Loss: 0.2238 Acc: 0.8742 F1: 0.6884 AUROC: 0.9257
100%|██████████| 297/297 [00:31<00:00,  9.47it/s]
INFO:root:val Loss: 0.0979 Acc: 0.9335 F1: 0.6406 AUROC: 0.9496
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 410/410 [01:41<00:00,  4.04it/s]
INFO:root:train Loss: 0.1085 Acc: 0.9467 F1: 0.8538 AUROC: 0.9819
100%|██████████| 297/297 [00:31<00:00,  9.52it/s]
INFO:root:val Loss: 0.1035 Acc: 0.9419 F1: 0.6734 AUROC: 0.9538
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 410/410 [01:41<00:00,  4.03it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 5
cfg['model']['backbone'] = 'resnet18'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 128
cfg['experiment_name'] = 'resnet18_02'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 205/205 [01:53<00:00,  1.81it/s]
INFO:root:train Loss: 0.3101 Acc: 0.8560 F1: 0.6233 AUROC: 0.8678
100%|██████████| 149/149 [00:40<00:00,  3.69it/s]
INFO:root:val Loss: 0.1075 Acc: 0.9337 F1: 0.6357 AUROC: 0.9495
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 205/205 [01:53<00:00,  1.81it/s]
INFO:root:train Loss: 0.1412 Acc: 0.9332 F1: 0.8176 AUROC: 0.9688
100%|██████████| 149/149 [00:39<00:00,  3.78it/s]
INFO:root:val Loss: 0.1061 Acc: 0.9413 F1: 0.6653 AUROC: 0.9478
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 205/205 [01:53<00:00,  1.81it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 5
cfg['model']['backbone'] = 'resnet18'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.2
cfg['dataloaders']['train_batch_size'] = 128
cfg['experiment_name'] = 'resnet18_03'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 205/205 [01:53<00:00,  1.81it/s]
INFO:root:train Loss: 0.2607 Acc: 0.8542 F1: 0.6419 AUROC: 0.8981
100%|██████████| 149/149 [00:40<00:00,  3.72it/s]
INFO:root:val Loss: 0.1003 Acc: 0.9327 F1: 0.6331 AUROC: 0.9482
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 205/205 [01:53<00:00,  1.81it/s]
INFO:root:train Loss: 0.1124 Acc: 0.9525 F1: 0.8649 AUROC: 0.9803
100%|██████████| 149/149 [00:40<00:00,  3.66it/s]
INFO:root:val Loss: 0.1354 Acc: 0.9270 F1: 0.6170 AUROC: 0.9484
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 205/205 [01:54<00:00,  1.80it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'resnet18'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 128
# cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'resnet18_04'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 362/362 [03:07<00:00,  1.93it/s]
INFO:root:train Loss: 0.2135 Acc: 0.8755 F1: 0.5287 AUROC: 0.8821
100%|██████████| 149/149 [00:40<00:00,  3.72it/s]
INFO:root:val Loss: 0.1017 Acc: 0.9321 F1: 0.6349 AUROC: 0.9490
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 362/362 [03:07<00:00,  1.93it/s]
INFO:root:train Loss: 0.0884 Acc: 0.9440 F1: 0.7556 AUROC: 0.9745
100%|██████████| 149/149 [00:40<00:00,  3.68it/s]
INFO:root:val Loss: 0.1089 Acc: 0.9361 F1: 0.6485 AUROC: 0.9430
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 362/362 [03:07<00:00,  1.93it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'resnet18'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.3
cfg['experiment_name'] = 'resnet18_04'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 362/362 [04:31<00:00,  1.33it/s]
INFO:root:train Loss: 0.2605 Acc: 0.8363 F1: 0.4161 AUROC: 0.8036
100%|██████████| 149/149 [00:40<00:00,  3.72it/s]
INFO:root:val Loss: 0.1175 Acc: 0.9091 F1: 0.5525 AUROC: 0.9328
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 362/362 [04:23<00:00,  1.37it/s]
INFO:root:train Loss: 0.1520 Acc: 0.9002 F1: 0.5985 AUROC: 0.9210
100%|██████████| 149/149 [00:39<00:00,  3.73it/s]
INFO:root:val Loss: 0.1173 Acc: 0.9447 F1: 0.6662 AUROC: 0.9391
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 362/362 [04:28<00:00,  1.35it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'resnet34'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 64
cfg['data_aug']['prob'] = 0
cfg['experiment_name'] = 'resnet34_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 724/724 [04:22<00:00,  2.76it/s]
INFO:root:train Loss: 0.1609 Acc: 0.8974 F1: 0.6049 AUROC: 0.9231
100%|██████████| 297/297 [00:44<00:00,  6.71it/s]
INFO:root:val Loss: 0.1000 Acc: 0.9375 F1: 0.6600 AUROC: 0.9481
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 724/724 [04:23<00:00,  2.75it/s]
INFO:root:train Loss: 0.0786 Acc: 0.9538 F1: 0.7928 AUROC: 0.9786
100%|██████████| 297/297 [00:43<00:00,  6.85it/s]
INFO:root:val Loss: 0.1192 Acc: 0.9277 F1: 0.6197 AUROC: 0.9419
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 724/724 [04:23<00:00,  2.75it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['dataset']['train_neg_to_pos_ratio'] =  20
cfg['model']['backbone'] = 'resnet34'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'resnet34_02'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
INFO:root:Epoch 1/15
INFO:root:----------
100%|██████████| 362/362 [05:37<00:00,  1.07it/s]
INFO:root:train Loss: 0.2544 Acc: 0.8575 F1: 0.4624 AUROC: 0.8185
100%|██████████| 149/149 [00:50<00:00,  2.94it/s]
INFO:root:val Loss: 0.1154 Acc: 0.9437 F1: 0.6660 AUROC: 0.9398
INFO:root:----------
INFO:root:Epoch 2/15
INFO:root:----------
100%|██████████| 362/362 

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'resnet50'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'resnet50_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/15
INFO:root:----------
100%|██████████| 362/362 [09:58<00:00,  1.65s/it]
INFO:root:train Loss: 0.1801 Acc: 0.8802 F1: 0.5425 AUROC: 0.8926
100%|██████████| 149/149 [01:14<00:00,  2.00it/s]
INFO:root:val Loss: 0.1242 Acc: 0.9503 F1: 0.6977 AUROC: 0.9417
INFO:root:----------
INFO:root:Epoch 2/15
INFO:root:----------
100%|██████████| 362/362 [09:50<00:00,  1.63s/it]
INFO:root:train Loss: 0.1130 Acc: 0.9206 F1: 0.6754 AUROC: 0.9576
100%|██████████| 149/149 [01:15<00:00,  1.97it/s]
INFO:root:val Loss: 0.1210 Acc: 0.9395 F1: 0.6540 AUROC: 0.9333
INFO:root:----------
INFO:root:Epoch 3/15
INFO:root:----------
100%|██████████| 362/362 [09:51<00:00,  1.63s/it]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'efficientnet_b0'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'efficientnet_b0_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/15
INFO:root:----------
100%|██████████| 362/362 [05:42<00:00,  1.06it/s]
INFO:root:train Loss: 0.2200 Acc: 0.8641 F1: 0.4690 AUROC: 0.8393
100%|██████████| 149/149 [00:48<00:00,  3.08it/s]
INFO:root:val Loss: 0.1203 Acc: 0.9281 F1: 0.5949 AUROC: 0.9188
INFO:root:----------
INFO:root:Epoch 2/15
INFO:root:----------
100%|██████████| 362/362 [05:25<00:00,  1.11it/s]
INFO:root:train Loss: 0.1370 Acc: 0.9111 F1: 0.6348 AUROC: 0.9324
100%|██████████| 149/149 [00:47<00:00,  3.12it/s]
INFO:root:val Loss: 0.1108 Acc: 0.9041 F1: 0.5469 AUROC: 0.9323
INFO:root:----------
INFO:root:Epoch 3/15
INFO:root:----------
100%|██████████| 362/362 [05:22<00:00,  1.12it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'resnet101'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'resnet101_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'resnet152'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'resnet152_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'efficientnonet_b1'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'efficientnet_b1_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'efficientnet_b2'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'efficientnet_b2_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'efficientnet_b3'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'efficientnet_b3_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'efficientnet_b4'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'efficientnet_b4_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'efficientnet_b5'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'efficientnet_b5_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'efficientnet_b6'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'efficientnet_b6_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'efficientnet_b7'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'efficientnet_b7_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()